In [1]:
import numpy as np
import networkx as nx
import scipy.sparse as sp
from random import randint

import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

from gensim.models import Word2Vec

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd

# INF 554 Lab 9: Deep Learning for Graph Data


We want to acknowledge _Dr. Giannis Nikolentzos_ for his large contributions to the content of this lab.

In this lab we will be working with the DeepWalk algorithm, an unsupervised method for the embedding of nodes in a  graph, and Graph Neural Networks, which are semi-supervised methods used for node classification and several other tasks on attributed graphs. Semi-supervised learning methods are trained on datasets, which include both labeled and unlabeled data points, and therefore are often said to fall inbetween supervised and unsupervised learning methods. 


As in the assessment we will be working with the Cora dataset, which we load below.



In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#cora_dataset = cora_dataset[0].to(device)
    
# load the graph
G = nx.read_edgelist('../data/coauthorship.edgelist', delimiter=' ', nodetype=int)
nodes = np.array(list(G.nodes))
#G = G.subgraph(nodes[:100])
#nodes = np.array(list(G.nodes))
adjacency = nx.linalg.graphmatrix.adjacency_matrix(G)
n = G.number_of_nodes()
m = G.number_of_edges()

# Read h indexes
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
print(df_train.shape)

indices = np.arange(110000)
indices = np.random.permutation(indices)
y_train = df_train['hindex'][indices[:100000]]
authors_train = df_train['author'][indices[:100000]]
y_test = df_train['hindex'][indices[-10000:]].to_numpy()
authors_test = df_train['author'][indices[-10000:]]
print(authors_train.shape, authors_test.shape)

G = G.subgraph(np.array([nodes[nodes==auth] for auth in authors_train] + [nodes[nodes==auth] for auth in authors_test]).reshape(len(y_train)+len(y_test)))
nodes = np.array(list(G.nodes))
adjacency = nx.convert_matrix.to_scipy_sparse_matrix(G)
N = np.load('../data/N.npy')
print(N[:10])
#features = np.expand_dims(np.copy(nodes), axis=1)
idx_train = np.array([np.argwhere(nodes==auth) for auth in authors_train])
idx_test = np.array([np.argwhere(nodes==auth) for auth in authors_test])
idx_train = idx_train.reshape(len(authors_train))
idx_test = idx_test.reshape(len(authors_test))
print(idx_train)
print(idx_test)
print(len(authors_train), len(authors_test))
print(np.min(idx_train), np.min(idx_test))
for i in idx_train:
    N[i,-1] = df_train[df_train['author'] == nodes[i]]['hindex']
for i in idx_test:
    N[i,-1] = -1.0
print(N[:10])
idx = np.concatenate((idx_train, idx_test))
features = N[idx]
n = G.number_of_nodes()
m = G.number_of_edges()
print(n, m)

(174241, 2)
(100000,) (10000,)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.18650612  0.26633653  0.20926507 ...  0.11414053 -0.02610091
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
[105831  83363  34553 ... 107648  83100  43012]
[25671 46599  5386 ... 91507  7006 39079]
100000 10000
0 15
[[ 0.          0.          0.         ...  0.          0.
  11.        ]
 [ 0.          0.          0.         ...  0.          0.
   2.        ]
 [ 0.          0.          0.         ...  0.          0.
   2.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   9.        ]
 [ 0.18650612  0.26633653  0.20926507 ...  0.11414053 -0.02610091
  14.        ]
 [ 0.          0.          0.  

In [24]:
#y_train = y_train.to_numpy()
#y_test = y_test.to_numpy()
print(y_train)
print(y_test)

26423      3.0
81322     17.0
89589     78.0
13372     15.0
48617      5.0
          ... 
29503      3.0
108588    16.0
16659      5.0
106192     4.0
12255      7.0
Name: hindex, Length: 100000, dtype: float64
[16.  8. 16. ...  7.  7.  3.]


In [25]:
print(np.min(nodes))

1515524


## 2) Graph Convolutional Networks

In the second part of the lab, we will focus on the problem of semi-supervised node classification using Graph Neural Networks (GNNs). Generally, speaking GNNs are neural networks that process graph data, in contrast to MLPs processing vector data, CNNs processing structured data such as images and RNNs processing sequences.

GNNs follow a recursive neighborhood aggregation (or message passing) scheme, where each node aggregates feature vectors of its neighbors to compute its new feature vector.
After $k$ iterations of aggregation, a node is represented by its transformed feature vector, which captures the structural information within the node's $k$-hop neighborhood.


In node classification, we are given the class labels of some nodes, and the goal is to predict the class labels of the nodes of the test set using information from both the graph structure and the attributes of the nodes.

Given the adjacency matrix $\mathbf{A}$ of a graph, we will first normalise it as follows:
\begin{equation*}
   \qquad\qquad\qquad\qquad \hat{\mathbf{A}} = (\mathbf{D}+\mathbf{I})^{-\frac{1}{2}} \; (\mathbf{A} +\mathbf{I}) \; (\mathbf{D}+\mathbf{I})^{-\frac{1}{2}}, \qquad\qquad\qquad\qquad(1)
\end{equation*}
where $\mathbf{D}$ is a diagonal matrix such that $\mathbf{D}_{ii} = \sum_j \mathbf{A}_{ij}$.
The above formula adds self-loops to the graph, and produces a symmetric normalised matrix.
This normalization trick, among other things, addresses numerical instabilities, which may lead to exploding/vanishing gradients when used in a deep neural network model.


>**Task 4:** Fill in the body of the ``normalise_adjacency()`` function that applies the normalisation trick in Equation (1). Note that the adjacency matrix is stored as a sparse matrix. Use operations of the NumPy and SciPy libraries (e.g., ``identity()`` function of SciPy) to also produce a sparse normalised matrix.    



In [26]:
def normalise_adjacency(A):
    #Please insert your code for Task 4 here
    diag = np.asarray(sp.csr_matrix.sum(A, axis=1)).squeeze()
    D = sp.diags(diag, format='csr')
    I = sp.identity(D.shape[0], format='csr')
    fac = sp.linalg.inv(sp.csr_matrix.sqrt(D + I))
    A_normalised = fac @ (A + I) @ fac
    return A_normalised


def sparse_to_torch_sparse(M):
    """Converts a sparse SciPy matrix to a sparse PyTorch tensor"""
    M = M.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((M.row, M.col)).astype(np.int64))
    values = torch.from_numpy(M.data)
    shape = torch.Size(M.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

print(type(adjacency))
#type(normalise_adjacency(sp.csr_matrix(adjacency)))

<class 'scipy.sparse.csr.csr_matrix'>


You will next implement a [Graph Convolutional Network (GCN)](https://arxiv.org/pdf/1609.02907.pdf) model that consists of three layers.
Let $\hat{\mathbf{A}}$ be the normalised adjacency matrix of the graph, and $\mathbf{X}$ a matrix whose $i^{th}$ row contains the feature vector of node $i$.
The first layer of the model is a message passing layer, and is defined as follows:
\begin{equation*}
    \mathbf{Z}^0 = f(\hat{\mathbf{A}} \; \mathbf{X} \; \mathbf{W}^0 ),
\end{equation*}
where $\mathbf{W}^0$ is a matrix of trainable weights and $f$ is an activation function (e.g., ReLU, sigmoid, tanh).
Clearly, the new feature vector of each node is the sum of the feature vectors of its neighbors.
The second layer of the model is again a message passing layer:
\begin{equation*}
    \mathbf{Z}^1 = f(\hat{\mathbf{A}} \; \mathbf{Z}^0 \; \mathbf{W}^1),
\end{equation*}
where $\mathbf{W}^1$ is a second matrix of trainable weights and $f$ is an activation function.
The two message passing layers are followed by a fully-connected layer which makes use of the softmax function to produce a probability distribution over the class labels:
\begin{equation*}
    \hat{\mathbf{Y}} = \text{softmax}(\mathbf{Z}^1 \; \mathbf{W}^2),
\end{equation*}
where $\mathbf{W}^2$ is a third matrix of trainable weights.
Note that for clarity of presentation we have omitted biases.

We next discuss some practical implementation details.
Let $\mathbf{H} = f(\hat{\mathbf{A}} \; \mathbf{Z} \; \mathbf{W}^0 )$ be a message passing layer.
Clearly, to compute $\mathbf{H}$, we need to perform two matrix-matrix multiplications.
Since $\mathbf{W}$ corresponds to a matrix of trainable parameters, the first matrix-matrix multiplication (between $\mathbf{Z}$ and $\mathbf{W}$) can be defined as a fully-connected layer of the network.
Then, we can multiply $\hat{\mathbf{A}}$ with the output of the above operation and apply the nonlinear function to compute $\mathbf{H}$.

>**Task 5:** Implement the architecture presented above in the ``GCN()`` class.
More specifically, add the following layers:
>* a message passing layer with $h_1$ hidden units (i.e., $\mathbf{W}^0 \in \mathbb{R}^{d \times h_1}$) followed by a ReLU activation function
>* a dropout layer with with $p_d$ ratio of dropped outputs
>* a message passing layer with $h_2$ hidden units (i.e., $\mathbf{W}^1 \in \mathbb{R}^{h_1 \times h_2}$) followed by a ReLU activation function
>* a fully-connected layer with $n_{class}$ units (i.e., $\mathbf{W}^2 \in \mathbb{R}^{h_2 \times n_{class}}$) followed by the log_softmax activation function

>Please return both the log_softmaxed output and the hidden states after the second layer, so that they can be visualised later on. Then, make use of our provided code to train this model on the cora dataset and visualise its hidden states.

>(Hint: You can perform a matrix-matrix multiplication using the ``torch.mm()`` function).


In [27]:

class GCN(nn.Module):
    """Simple GCN model"""
    def __init__(self, n_feat, n_hidden_1, n_hidden_2, dropout):
        super(GCN, self).__init__()

        self.fc1 = nn.Linear(n_feat, n_hidden_1)
        self.fc2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.fc3 = nn.Linear(n_hidden_2, 1)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

    def forward(self, x_in, adj):
        
        #Please insert your code for Task 5 here
        z0 = self.relu(self.fc1(adj.matmul(torch.tensor(x_in, dtype=torch.float32))))
        z0 = self.dropout(z0)
        z1 = self.relu(self.fc2(adj @ z0))
        x = self.fc3(z1)

        return x, z1

In [10]:
adj = normalise_adjacency(sp.csr_matrix(adjacency)) 
adj = sparse_to_torch_sparse(sp.csr_matrix(adjacency)).to(device)

/home/erwan/.local/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/erwan/.local/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


In [28]:
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [34]:
# Hyperparameters
epochs = 500
n_hidden_1 = 256
n_hidden_2 = 32
learning_rate = 1e-2
dropout_rate = 0.5


# Creates the model and specifies the optimizer
model = GCN(features.shape[1], n_hidden_1, n_hidden_2, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss = nn.MSELoss()

def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output,_ = model(features, adj)
    loss_train = loss(output[idx_train].reshape(len(idx_train)), y_train)
    acc_train = accuracy_score(torch.argmax(output[idx_train], dim=1).detach().cpu().numpy(), y_train.cpu().numpy())
    loss_train.backward()
    optimizer.step()

    
    model.eval()
    output,_ = model(features, adj)

    loss_val = loss(output[idx_test].reshape(len(idx_test)), y_test)
    acc_val = accuracy_score(torch.argmax(output[idx_test], dim=1).detach().cpu().numpy(), y_test.cpu().numpy())
    print('Epoch: {:03d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    model.eval()
    output, embeddings = model(features, adj)
    loss_test = loss(output[idx_test].reshape(len(idx_test)), y_test)
    acc_test = accuracy_score(torch.argmax(output[idx_test], dim=1).detach().cpu().numpy(), y_test.cpu().numpy())
    
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test))

    return embeddings

# Train model
print('Begin training...')
t_total = time.time()
for epoch in range(epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
print()

# Testing
GCN_embeddings = test()

Begin training...
Epoch: 001 loss_train: 224.9492 acc_train: 0.0000 loss_val: 224.3655 acc_val: 0.0000 time: 1.9505s
Epoch: 002 loss_train: 223.6556 acc_train: 0.0000 loss_val: 228.4854 acc_val: 0.0000 time: 1.7091s
Epoch: 003 loss_train: 223.8230 acc_train: 0.0000 loss_val: 230.6032 acc_val: 0.0000 time: 1.7410s
Epoch: 004 loss_train: 224.7843 acc_train: 0.0000 loss_val: 230.2570 acc_val: 0.0000 time: 1.9321s
Epoch: 005 loss_train: 225.6424 acc_train: 0.0000 loss_val: 228.2746 acc_val: 0.0000 time: 1.9787s
Epoch: 006 loss_train: 223.6491 acc_train: 0.0000 loss_val: 224.5224 acc_val: 0.0000 time: 1.8510s
Epoch: 007 loss_train: 222.6242 acc_train: 0.0000 loss_val: 219.6008 acc_val: 0.0000 time: 1.8602s
Epoch: 008 loss_train: 229.8526 acc_train: 0.0000 loss_val: 232.2337 acc_val: 0.0000 time: 1.7494s
Epoch: 009 loss_train: 227.3819 acc_train: 0.0000 loss_val: 240.9110 acc_val: 0.0000 time: 1.7210s
Epoch: 010 loss_train: 230.9195 acc_train: 0.0000 loss_val: 244.9938 acc_val: 0.0000 time: 

Epoch: 084 loss_train: 218.9583 acc_train: 0.0000 loss_val: 223.1877 acc_val: 0.0000 time: 1.8053s
Epoch: 085 loss_train: 219.5648 acc_train: 0.0000 loss_val: 221.0125 acc_val: 0.0000 time: 1.7546s
Epoch: 086 loss_train: 221.0573 acc_train: 0.0000 loss_val: 218.9377 acc_val: 0.0000 time: 1.7089s
Epoch: 087 loss_train: 219.3339 acc_train: 0.0000 loss_val: 222.2783 acc_val: 0.0000 time: 1.6354s
Epoch: 088 loss_train: 219.8356 acc_train: 0.0000 loss_val: 224.8689 acc_val: 0.0000 time: 1.6325s
Epoch: 089 loss_train: 221.4472 acc_train: 0.0000 loss_val: 225.2429 acc_val: 0.0000 time: 1.7115s
Epoch: 090 loss_train: 221.1520 acc_train: 0.0000 loss_val: 224.1464 acc_val: 0.0000 time: 1.9375s
Epoch: 091 loss_train: 219.1220 acc_train: 0.0000 loss_val: 220.2904 acc_val: 0.0000 time: 1.8273s
Epoch: 092 loss_train: 219.8068 acc_train: 0.0000 loss_val: 216.6062 acc_val: 0.0000 time: 1.6997s
Epoch: 093 loss_train: 220.8568 acc_train: 0.0000 loss_val: 221.1796 acc_val: 0.0000 time: 1.4481s
Epoch: 094

Epoch: 167 loss_train: 220.7253 acc_train: 0.0000 loss_val: 219.0495 acc_val: 0.0000 time: 1.7969s
Epoch: 168 loss_train: 215.9820 acc_train: 0.0000 loss_val: 211.9776 acc_val: 0.0000 time: 1.7024s
Epoch: 169 loss_train: 240.1435 acc_train: 0.0000 loss_val: 234.1701 acc_val: 0.0000 time: 1.9039s
Epoch: 170 loss_train: 227.2797 acc_train: 0.0000 loss_val: 247.7998 acc_val: 0.0000 time: 1.9998s
Epoch: 171 loss_train: 238.0425 acc_train: 0.0000 loss_val: 253.4273 acc_val: 0.0000 time: 2.0061s
Epoch: 172 loss_train: 242.7414 acc_train: 0.0000 loss_val: 255.2662 acc_val: 0.0000 time: 1.9742s
Epoch: 173 loss_train: 244.8840 acc_train: 0.0000 loss_val: 255.1846 acc_val: 0.0000 time: 1.9542s
Epoch: 174 loss_train: 243.5557 acc_train: 0.0000 loss_val: 254.0665 acc_val: 0.0000 time: 1.9284s
Epoch: 175 loss_train: 244.0533 acc_train: 0.0000 loss_val: 252.1124 acc_val: 0.0000 time: 2.0398s
Epoch: 176 loss_train: 242.5779 acc_train: 0.0000 loss_val: 249.4178 acc_val: 0.0000 time: 1.8817s
Epoch: 177

Epoch: 250 loss_train: 223.6787 acc_train: 0.0000 loss_val: 221.1017 acc_val: 0.0000 time: 1.9410s
Epoch: 251 loss_train: 224.6094 acc_train: 0.0000 loss_val: 220.3677 acc_val: 0.0000 time: 1.8873s
Epoch: 252 loss_train: 226.7854 acc_train: 0.0000 loss_val: 220.1692 acc_val: 0.0000 time: 1.7566s
Epoch: 253 loss_train: 224.2514 acc_train: 0.0000 loss_val: 219.9637 acc_val: 0.0000 time: 1.8540s
Epoch: 254 loss_train: 228.5865 acc_train: 0.0000 loss_val: 226.9635 acc_val: 0.0000 time: 1.8839s
Epoch: 255 loss_train: 227.0460 acc_train: 0.0000 loss_val: 231.3051 acc_val: 0.0000 time: 1.9851s
Epoch: 256 loss_train: 227.3246 acc_train: 0.0000 loss_val: 233.2226 acc_val: 0.0000 time: 1.8298s
Epoch: 257 loss_train: 228.9275 acc_train: 0.0000 loss_val: 233.7478 acc_val: 0.0000 time: 1.9039s
Epoch: 258 loss_train: 230.1133 acc_train: 0.0000 loss_val: 233.3367 acc_val: 0.0000 time: 1.7910s
Epoch: 259 loss_train: 231.4871 acc_train: 0.0000 loss_val: 232.1754 acc_val: 0.0000 time: 1.8291s
Epoch: 260

Epoch: 333 loss_train: 228.0537 acc_train: 0.0000 loss_val: 218.8140 acc_val: 0.0000 time: 1.7040s
Epoch: 334 loss_train: 221.0452 acc_train: 0.0000 loss_val: 224.6430 acc_val: 0.0000 time: 1.7412s
Epoch: 335 loss_train: 222.7742 acc_train: 0.0000 loss_val: 228.6420 acc_val: 0.0000 time: 1.9141s
Epoch: 336 loss_train: 225.8670 acc_train: 0.0000 loss_val: 230.8076 acc_val: 0.0000 time: 1.8807s
Epoch: 337 loss_train: 227.2422 acc_train: 0.0000 loss_val: 231.5358 acc_val: 0.0000 time: 1.8827s
Epoch: 338 loss_train: 227.6932 acc_train: 0.0000 loss_val: 231.2684 acc_val: 0.0000 time: 1.9323s
Epoch: 339 loss_train: 227.9356 acc_train: 0.0000 loss_val: 230.1118 acc_val: 0.0000 time: 1.8226s
Epoch: 340 loss_train: 226.7055 acc_train: 0.0000 loss_val: 227.9421 acc_val: 0.0000 time: 2.1219s
Epoch: 341 loss_train: 224.9820 acc_train: 0.0000 loss_val: 224.8085 acc_val: 0.0000 time: 2.2602s
Epoch: 342 loss_train: 222.9247 acc_train: 0.0000 loss_val: 220.6010 acc_val: 0.0000 time: 1.8484s
Epoch: 343

Epoch: 416 loss_train: 217.8719 acc_train: 0.0000 loss_val: 213.8882 acc_val: 0.0000 time: 1.8607s
Epoch: 417 loss_train: 216.9557 acc_train: 0.0000 loss_val: 213.4088 acc_val: 0.0000 time: 1.7293s
Epoch: 418 loss_train: 217.1376 acc_train: 0.0000 loss_val: 212.9779 acc_val: 0.0000 time: 1.9402s
Epoch: 419 loss_train: 217.6504 acc_train: 0.0000 loss_val: 212.7615 acc_val: 0.0000 time: 1.9859s
Epoch: 420 loss_train: 217.7086 acc_train: 0.0000 loss_val: 212.4994 acc_val: 0.0000 time: 1.9704s
Epoch: 421 loss_train: 215.9342 acc_train: 0.0000 loss_val: 212.2685 acc_val: 0.0000 time: 1.9127s
Epoch: 422 loss_train: 218.0382 acc_train: 0.0000 loss_val: 212.4765 acc_val: 0.0000 time: 1.8110s
Epoch: 423 loss_train: 216.7388 acc_train: 0.0000 loss_val: 212.9423 acc_val: 0.0000 time: 1.8603s
Epoch: 424 loss_train: 215.3428 acc_train: 0.0000 loss_val: 212.9619 acc_val: 0.0000 time: 1.8648s
Epoch: 425 loss_train: 214.3826 acc_train: 0.0000 loss_val: 212.3537 acc_val: 0.0000 time: 1.9264s
Epoch: 426

Epoch: 499 loss_train: 220.3937 acc_train: 0.0000 loss_val: 213.1201 acc_val: 0.0000 time: 1.8664s
Epoch: 500 loss_train: 213.4197 acc_train: 0.0000 loss_val: 217.9125 acc_val: 0.0000 time: 1.9271s
Optimization Finished!
Total time elapsed: 906.4544s

Test set results: loss= 217.9125 accuracy= 0.0000



We will now visualise the hidden states, which are fed into the final layer of the GCN architecture.

In [ ]:
GCN_embeddings_local = GCN_embeddings.detach().cpu().numpy()


my_pca = PCA(n_components=10)
my_tsne = TSNE(n_components=2)

vecs_pca = my_pca.fit_transform(GCN_embeddings_local)
vecs_tsne = my_tsne.fit_transform(vecs_pca)

plt.figure(figsize=(7,7))
plt.title("TSNE visualisation of the GCN embeddings")
colours = sns.color_palette("hls", len(np.unique(class_labels)))
sns.scatterplot(x=vecs_tsne[:,0], y=vecs_tsne[:,1], hue=class_labels, legend='full', palette=colours)


plt.show()

This concludes the taught part of the INF554 course. In the past weeks we had the pleasure to meet a wide range of machine and deep learning models ranging from linear models all the way to graph neural networks. For further context on what we have learned, we want to leave you with a famous quote of George Box:

> _"All models are wrong, but some are useful."_

In [35]:
torch.save(model.state_dict(), 'trained_models/model_2.pt')

the_model = GCN(features.shape[1], n_hidden_1, n_hidden_2, dropout_rate).to(device)
the_model.load_state_dict(torch.load('trained_models/model_2.pt'))

<All keys matched successfully>

In [13]:
print(y_train.shape[0] + y_test.shape[0])
print(features.shape[0])

110000
110000


In [16]:
G2 = nx.read_edgelist('../data/coauthorship.edgelist', delimiter=' ', nodetype=int)
nodes = np.array(list(G2.nodes))
print(nodes.shape)
print(df_train.shape)
print(df_test.shape)
print(df_train.shape[0] + df_test.shape[0])

(217801,)
(174241, 2)
(43560, 3)
217801


In [33]:
model.eval()
output,_ = model(features, adj)
t = output[idx_test][-10:]
a = nodes[idx_test][-10:]
v = y_test[-10:]
print(max(y_test))
print(t)
print(a)
print(v)

tensor(135.)
tensor([[1.5785],
        [4.3889],
        [7.9216],
        [3.9266],
        [7.5771],
        [6.0223],
        [4.7441],
        [3.2470],
        [0.8580],
        [8.4155]], grad_fn=<SliceBackward0>)
[2176915274 1966933645  223937459 1983936918 2107460205 1964615562
 2063529815 1232375858 1988909771 1996728517]
tensor([8., 8., 2., 3., 2., 7., 4., 7., 7., 3.])
